In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# Define directories
dataset_dir = 'Dataset'
augmented_dir = 'augmented_images'
os.makedirs(augmented_dir, exist_ok=True)

train_dir = os.path.join(augmented_dir, 'training')
val_dir = os.path.join(augmented_dir, 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


In [3]:
# Initialize ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
def augment_and_save_images(source_dir, target_dir, num_images_to_generate):
    categories = ['yes', 'no']
    for category in categories:
        category_dir = os.path.join(source_dir, category)
        target_category_dir = os.path.join(target_dir, category)
        
        # Check if source directory exists
        if not os.path.exists(category_dir):
            print(f"Source directory {category_dir} does not exist.")
            continue
        
        # Create target directory if it does not exist
        if not os.path.exists(target_category_dir):
            os.makedirs(target_category_dir)
        
        image_files = [os.path.join(category_dir, fname) for fname in os.listdir(category_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]
        counter = 0
        
        for image_path in image_files:
            try:
                image = load_img(image_path)
                image_array = img_to_array(image)
                image_array = np.expand_dims(image_array, axis=0)
                
                for batch in datagen.flow(image_array, batch_size=1):
                    augmented_image = array_to_img(batch[0])
                    augmented_image.save(os.path.join(target_category_dir, f'{category}_augmented_{counter:04d}.jpg'))
                    counter += 1
                    if counter >= num_images_to_generate:
                        break
                if counter >= num_images_to_generate:
                    break
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

# Number of images to generate for training and validation sets
num_images_per_category = 1000  # Adjust as needed

# Augment and save images
augment_and_save_images(dataset_dir, train_dir, num_images_per_category)
augment_and_save_images(dataset_dir, val_dir, num_images_per_category)


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    augmented_dir + '/training',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    augmented_dir + '/validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


Retrain the Model